In [25]:
import pandas as pd
import numpy as np
import json
from pandas.io.json import json_normalize
import os
import glob
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
from scipy.stats import chi2_contingency
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import OneHotEncoder
from tqdm import tqdm
from sklearn.metrics import accuracy_score, confusion_matrix, roc_curve, roc_auc_score

In [32]:
Predictors = pd.read_csv('Predictors.csv')
target_1 = pd.read_csv('target_1.csv')

In [33]:
target_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 546420 entries, 0 to 546419
Data columns (total 4 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   Unnamed: 0      546420 non-null  int64  
 1   target          546420 non-null  float64
 2   EquipmentID     546420 non-null  int64  
 3   EventTimeStamp  546420 non-null  object 
dtypes: float64(1), int64(2), object(1)
memory usage: 16.7+ MB


In [34]:
target_1 = target_1.drop(columns = 'Unnamed: 0')

In [35]:
target_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 546420 entries, 0 to 546419
Data columns (total 3 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   target          546420 non-null  float64
 1   EquipmentID     546420 non-null  int64  
 2   EventTimeStamp  546420 non-null  object 
dtypes: float64(1), int64(1), object(1)
memory usage: 12.5+ MB


In [38]:
trucks = target_1[['EquipmentID']].drop_duplicates()

derates = target_1.loc[target_1['target'] == 1]['EquipmentID'].drop_duplicates().to_list()

trucks['derate'] = np.where(
    trucks['EquipmentID'].isin(derates), 1, 0
)

xTrain, xTest, yTrain, yTest = train_test_split(
    trucks['EquipmentID'], trucks['EquipmentID'], test_size = .2, random_state = 777, stratify = trucks['derate']
)

xTrain = (Predictors.loc[Predictors['EquipmentID'].isin(xTrain)]
          .drop(columns = ['EquipmentID'])
          .reset_index(drop = True)
         )
xTest = (Predictors.loc[Predictors['EquipmentID'].isin(xTest)]
          .drop(columns = ['EquipmentID'])
          .reset_index(drop = True)
         )
yTrain = target_1.loc[target_1['EquipmentID'].isin(yTrain)].reset_index(drop = True)
yTest = target_1.loc[target_1['EquipmentID'].isin(yTest)].reset_index(drop = True)

In [39]:
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import RandomOverSampler, SMOTE, ADASYN
from pycaret.datasets import get_data
from sklearn.decomposition import PCA
from pycaret.classification import *
import xgboost as xgb
from sklearn.metrics import confusion_matrix


In [40]:
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', xgb.XGBClassifier(n_estimators=1000, learning_rate=0.05, max_depth=5, random_state=42, scale_pos_weight=844))
])

pipeline.fit(xTrain, yTrain['target'] == 1)

Pipeline(steps=[('scaler', StandardScaler()),
                ('classifier',
                 XGBClassifier(base_score=None, booster=None, callbacks=None,
                               colsample_bylevel=None, colsample_bynode=None,
                               colsample_bytree=None,
                               early_stopping_rounds=None,
                               enable_categorical=False, eval_metric=None,
                               feature_types=None, gamma=None, gpu_id=None,
                               grow_policy=None, importance_type=None,
                               interaction_constraints=None, learning_rate=0.05,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=5, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, n_estimators=1000,
                               n_jobs=None, num_parallel_tree=None,
                               predictor=None, random_state=42, ...))])

In [45]:
importances = pd.DataFrame({
    'variable': xTest.columns.to_list(),
    'importance': pipeline['classifier'].feature_importances_
})

importances.sort_values('importance', ascending = False)

,variable,importance
992,FuelTemperature,0.127470
816,spnFmi_74-14,0.077392
193,spnFmi_1761-18,0.062564
280,spnFmi_3031-9,0.057464
192,spnFmi_1761-17,0.050993
...,...,...
407,spnFmi_3556-1,0.000000
408,spnFmi_3556-12,0.000000
409,spnFmi_3556-18,0.000000
410,spnFmi_3556-2,0.000000


In [46]:
yTest['prediction'] = pipeline.predict_proba(xTest)[:, 1]

yTrain['prediction'] = pipeline.predict_proba(xTrain)[:, 1]

In [26]:
yTest

,target,EquipmentID,EventTimeStamp,prediction
0,0.0,302,2010-12-31 23:04:15,0.002143
1,0.0,302,2015-02-21 12:52:42,0.000194
2,0.0,302,2015-02-21 12:52:42,0.000194
3,0.0,302,2015-02-21 15:40:57,0.000244
4,0.0,302,2015-02-21 15:40:57,0.000244
...,...,...,...,...
135599,0.0,2362,2020-03-06 09:36:42,0.000460
135600,0.0,2363,2019-12-08 11:09:48,0.000609
135601,0.0,2363,2019-12-20 07:27:06,0.000313
135602,0.0,2363,2020-01-15 22:16:34,0.002613


In [35]:
yTrain = pd.DataFrame(yTrain)

In [47]:
yTrain.to_csv('E:/Data Science/NSS_Projects/big_g_express-data-detectives/yTrain2.csv')

In [48]:
yTest.to_csv('E:/Data Science/NSS_Projects/big_g_express-data-detectives/yTest2.csv')

In [ ]:
# Set up the environment and create a model
env = setup(data=xTrain, target=yTrain, preprocess=['RandomOverSampler', 'StandardScaler'])
model = create_model('xgboost', use_gpu=True)

# Evaluate the model
evaluate_model(model)

# Make predictions with the model
predictions = predict_model(model, data=xTest)

,Description,Value
0,Session id,6741
1,Target,ecuMake_unknown
2,Target type,Binary
3,Original data shape,"(446856, 1010)"
4,Transformed data shape,"(446856, 1010)"
5,Transformed train set shape,"(312799, 1010)"
6,Transformed test set shape,"(134057, 1010)"
7,Numeric features,1009
8,Preprocess,"['RandomOverSampler', 'StandardScaler']"
9,Imputation type,simple


,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,13:48:19
Status,. . . . . . . . . . . . . . . . . .,Fitting 10 Folds
Estimator,. . . . . . . . . . . . . . . . . .,Extreme Gradient Boosting


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
# Evaluate the predictions
from sklearn.metrics import classification_report
print(classification_report(yTest, predictions['Label']))

In [ ]:
# Define the oversampling functions you want to use
oversampling = RandomOverSampler()


# Define the machine learning model you want to use
classifier = RandomForestClassifier()

# Create the pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('oversampling', oversampling),
    ('classifier', classifier)
])

# Fit the pipeline to your training data
pipeline.fit(xTrain, yTrain['target']==1)

score = pipeline.score(xTest, yTest['target']==1)
print("Accuracy:", score)

# Make predictions on your test data
y_pred = pipeline.predict(xTest)